In [1]:
# Python 2/3 compatibility
from __future__ import print_function

import numpy as np
import cv2
import video


In [2]:
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (x2, y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr


def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv2.remap(img, flow, None, cv2.INTER_LINEAR)
    return res

In [3]:
rows = 250
cols = 250

rec_sizex = 4
rec_sizey = 50
rec_posx = 1
rec_posy = 100


gray = np.zeros((rows,cols),np.uint8)
cv2.rectangle(gray,(rec_posx,rec_posy),(rec_posx+rec_sizex,rec_posy+rec_sizey),255,-1)
rec_posx+= 10


show_hsv = True
show_glitch = True
cur_glitch = gray.copy()

cv2.startWindowThread()
cv2.namedWindow("flow")
cv2.namedWindow("flow HSV")
cv2.namedWindow("glitch")


while (rec_posx < cols-rec_sizex):
    prevgray = gray

    gray = np.zeros((rows,cols),np.uint8)
    cv2.rectangle(gray,(rec_posx,rec_posy),(rec_posx+rec_sizex,rec_posy+rec_sizey),255,-1)
    rec_posx+=10

    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    cv2.imshow('flow', draw_flow(gray, flow))
    cv2.imshow('flow HSV', draw_hsv(flow))
    cur_glitch = warp_flow(cur_glitch, flow)
    cv2.imshow('glitch', cur_glitch)

    ch = cv2.waitKey(0)
    if ch == 27:
        break
cv2.destroyAllWindows()